<a href="https://colab.research.google.com/github/Mahdi-Saadati/Advanced-Machine-Learning-and-Deep-Neural-Networks/blob/main/5.a.%20Save%20the%20Trained%20Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl" align="right">

پروژه تشخیص افراد سالم و افراد دارای ذات الریه از روی تصاویر پزشکی قفسه سینه به همراه ذخیره مدل آموزش دیده شده

یکی از دیتاست‌های رایج و آزاد در حوزه پزشکی، دیتاست Chest X-Ray (Pneumonia) هست که شامل تصاویر سالم (Normal) و مبتلا به ذات‌الریه (Pneumonia) میباشد.
دیتاست را میتوانید از لینک زیر دانلود کنید:

https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia

In [ ]:
# 1. وارد کردن کتابخانه‌ها
import tensorflow as tf  # برای ساخت و آموزش مدل‌های یادگیری عمیق
from keras import layers, models  # برای ساخت لایه‌ها و مدل‌های شبکه عصبی

# 2. مسیر پوشه‌های داده
train_dir = "chest_xray/train"  # تصاویر آموزش
val_dir   = "chest_xray/val"    # تصاویر اعتبارسنجی
test_dir  = "chest_xray/test"   # تصاویر تست

# 3. تعریف پارامترها
IMG_SIZE = 150   # اندازه تصاویر: 150x150 پیکسل
BATCH_SIZE = 32  # تعداد تصاویر در هر بسته (batch)

# 4. ساخت Dataset آموزش
train_ds = tf.keras.utils.image_dataset_from_directory(
    train_dir,               # مسیر داده‌ها
    image_size=(IMG_SIZE, IMG_SIZE),  # تغییر اندازه تصاویر
    batch_size=BATCH_SIZE,           # تعداد تصاویر در هر batch
    label_mode='binary',             # برچسب‌ها دودویی: Normal=0 یا Pneumonia=1
    shuffle=True                     # تصاویر به صورت تصادفی مرتب شوند
)

# 5. ساخت Dataset اعتبارسنجی
val_ds = tf.keras.utils.image_dataset_from_directory(
    val_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# 6. ساخت Dataset تست
test_ds = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    label_mode='binary'
)

# 7. نرمال‌سازی تصاویر (پیکسل‌ها از 0-255 به 0-1 تبدیل شوند)
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds   = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds  = test_ds.map(lambda x, y: (normalization_layer(x), y))

# 8. ساخت مدل CNN
model = models.Sequential([
    # لایه کانولوشن 32 فیلتر 3x3 + فعال‌سازی ReLU
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    layers.MaxPooling2D(2,2),  # کاهش ابعاد با max pooling 2x2

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),            # تبدیل داده‌ها به یک بردار 1 بعدی
    layers.Dense(128, activation='relu'),  # لایه Dense با 128 نورون
    layers.Dropout(0.5),        # کاهش overfitting با حذف تصادفی 50٪ نورون‌ها
    layers.Dense(1, activation='sigmoid')  # خروجی: احتمال Pneumonia
])

# 9. کامپایل مدل
model.compile(
    optimizer='adam',                 # بهینه‌ساز Adam
    loss='binary_crossentropy',       # تابع خطای دودویی
    metrics=['accuracy']              # دقت مدل
)

# 10. آموزش مدل
history = model.fit(
    train_ds,           # داده‌های آموزش
    validation_data=val_ds,  # داده‌های اعتبارسنجی
    epochs=10           # تعداد دورهای آموزش
)

# 11. ارزیابی مدل روی داده تست
loss, acc = model.evaluate(test_ds)
print(f"Test Accuracy: {acc*100:.2f}%")  # نمایش دقت

# 12. ذخیره مدل آموزش دیده
model.save("chest_xray_cnn_model_tf.keras.h5")
